In [1]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
from nipype.interfaces import fsl
import nipype.interfaces.mrtrix3 as mtx
import nipype.interfaces.freesurfer as fsr
from pandas import Series, read_csv, to_numeric
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove, getcwd, makedirs
from shutil import copyfile
from nipype import config, logging
from datetime import date
today = str(date.today())
config.enable_debug_mode()


In [15]:
# Set variables
user = expanduser('~')
if user == '/Users/lucindasisk':
    home = join(user, 'Desktop/Milgram/candlab')
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    workflow_dir = join(home, 'analyses/shapes/dwi/preproc_workflow')
    data_dir = join(home, 'analyses/shapes/dwi/data')
else:
    home = '/gpfs/milgram/project/gee_dylan/candlab'
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    workflow_dir = join(home, 'analyses/shapes/dwi/preproc_workflow')
    data_dir = join(home, 'analyses/shapes/dwi/data')
    
# Read in subject subject_list
# subject_csv = read_csv(home + '/scripts/shapes/mri/dwi/shapes_dwi_subjList_08.07.2019.txt', sep=' ', header=None)
# subject_list = subject_csv[0].values.tolist()

# Manual subject list
subject_list = ['sub-A200', 'sub-A201']


['sub-A200',
 'sub-A201',
 'sub-A202',
 'sub-A204',
 'sub-A206',
 'sub-A207',
 'sub-A208',
 'sub-A210',
 'sub-A211',
 'sub-A212',
 'sub-A213',
 'sub-A214',
 'sub-A215',
 'sub-A216',
 'sub-A217',
 'sub-A218',
 'sub-A222',
 'sub-A223',
 'sub-A225',
 'sub-A228',
 'sub-A229',
 'sub-A230',
 'sub-A231',
 'sub-A232',
 'sub-A233',
 'sub-A234',
 'sub-A235',
 'sub-A236',
 'sub-A237',
 'sub-A238',
 'sub-A240',
 'sub-A241',
 'sub-A242',
 'sub-A246',
 'sub-A247',
 'sub-A248',
 'sub-A250',
 'sub-A251',
 'sub-A253',
 'sub-A255',
 'sub-A256',
 'sub-A257',
 'sub-A258',
 'sub-A260',
 'sub-A261',
 'sub-A262',
 'sub-A266',
 'sub-A268',
 'sub-A270',
 'sub-A271',
 'sub-A272',
 'sub-A273',
 'sub-A276',
 'sub-A279',
 'sub-A280',
 'sub-A281',
 'sub-A283',
 'sub-A285',
 'sub-A286',
 'sub-A288',
 'sub-A291',
 'sub-A293',
 'sub-A294',
 'sub-A328',
 'sub-A348',
 'sub-A351',
 'sub-A363',
 'sub-A374',
 'sub-A383',
 'sub-A401',
 'sub-A415',
 'sub-A417',
 'sub-A420',
 'sub-A427',
 'sub-A429',
 'sub-A437',
 'sub-A438',

In [9]:
# 9/22/19: change so that T1 is registered to B0 per https://mrtrix.readthedocs.io/en/latest/quantitative_structural_connectivity/act.html

In [10]:
# Create preprocessing Workflow

# set default FreeSurfer subjects dir
fsr.FSCommand.set_default_subjects_dir(raw_dir)

# Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory=data_dir,
                         substitutions=[('_subject_id_', '')]),
                name='datasink')

# Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

# SelectFiles
template = dict(t1=join(raw_dir, '{subject_id}/ses-shapesV1/anat/{subject_id}_ses-shapesV1_T1w.nii.gz'),
                dti=join(
                    raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.nii.gz'),
                bval=join(
                    raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.bval'),
                bvec=join(
                    raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.bvec'),
                fmappa=join(
                    raw_dir, '{subject_id}/ses-shapesV1/fmap/{subject_id}_ses-shapesV1_acq-dwi_dir-PA_epi.nii.gz'),
                fmapap=join(
                    raw_dir, '{subject_id}/ses-shapesV1/fmap/{subject_id}_ses-shapesV1_acq-dwi_dir-AP_epi.nii.gz'),
                aps=join(raw_dir, 'shapes_acqparams.txt'),
                index=join(raw_dir, 'shapes_index.txt'),
                mni=join(home, 'atlases/MNI152_T1_2mm_brain.nii.gz')
                )

sf = Node(SelectFiles(template,
                      base_directory=home),
          name='sf')



In [11]:
# Merge AP/PA encoding direction fieldmaps
def create_merged_files(ap, pa):
    from nipype.interfaces import fsl
    from os.path import abspath
    merge = fsl.Merge(in_files=[ap, pa],
                      dimension='t', output_type='NIFTI_GZ', merged_file='AP_PA_merged.nii.gz').run()
    merged_file = abspath('AP_PA_merged.nii.gz')
    return merged_file


create_merge = Node(Function(input_names=['ap', 'pa'],
                             output_names=['merged_file'],
                             function=create_merged_files),
                    name='create_merge')

In [12]:
# Resample T1w to same voxel dimensions as DTI to avoid data interpolation (1.714286 x 1.714286 x 1.700001) .
resamp_1 = Node(fsr.Resample(voxel_size=(1.7, 1.7, 1.7)),
                name='resamp_1')

resamp_2 = Node(fsr.Resample(voxel_size=(1.7, 1.7, 1.7)),
                name='resamp_2')

# Drop bottom slice (S/I) to create even # of slices
drop = Node(fsl.ExtractROI(x_min=0, x_size=140,
                           y_min=0, y_size=140,
                           z_min=1, z_size=80, output_type='NIFTI_GZ'),
            name='drop')

# drop bottom slice of DTI file
drop2 = drop.clone(name='drop2')

# Denoise DWI data susing local PCA correction - mrTrix3
denoise = Node(mtx.DWIDenoise(out_file='denoised.nii.gz'),
               name='denoise')

# Steps added 7/17 per Jiook's reccomendations
# Gibbs ringing removal
gibbs = Node(mtx.MRDeGibbs(out_file='denoised_gibbs.nii.gz'),
             name='gibbs')

# DWI bias file correction using ANTS N4
bias = Node(mtx.DWIBiasCorrect(use_ants=True,
                               out_file='denoised_gibbs_bias.nii.gz'),
            name='bias')

###########################

# Run topup on merged files from pe1 and pe0
topup = Node(fsl.TOPUP(config='b02b0.cnf',
                       out_corrected='ap_pa_topup.nii.gz', output_type='NIFTI_GZ'),
             name='topup')

# Select b0 image for registration
fslroi = Node(fsl.ExtractROI(t_min=0,
                             t_size=1,
                             roi_file='b0_img.nii.gz', output_type='NIFTI_GZ'),
              name='fslroi')

# Reorient topup b0 image to std
reorient1 = Node(fsl.Reorient2Std(output_type='NIFTI_GZ'),
                 name='reorient1')

# Register T1 to b0 - rigid 2D transformation
register1 = Node(fsl.FLIRT(out_matrix_file='b0toT1_reorient_reg.mat',
                           rigid2D=True,
                           output_type='NIFTI_GZ'),
                 name='register1')

# apply topup from merged file to rest of pe0 scan
apptop = Node(fsl.ApplyTOPUP(method='jac',
                             in_index=[2], 
                             output_type='NIFTI_GZ',
                            out_corrected = 'preprocessed_dwi.nii.gz'),
              name='apptop')

# Skullstrip the T1w image
stripT1 = Node(fsl.BET(mask=True, output_type='NIFTI_GZ'),
               name='stripT1')

#Eddy_CUDA Node
# FSL Eddy correction to remove eddy current distortion

eddy = Node(fsl.Eddy(is_shelled=True,
                     interp='trilinear',
                     method='jac',
                     output_type='NIFTI_GZ',
                     residuals=True,
                     use_cuda=True,
                     cnr_maps=True,
                     repol=True),
            name='eddy')



In [13]:

preproc_flow = Workflow(name='preproc_flow')
preproc_flow.connect([(infosource, sf, [('subject_id', 'subject_id')]),
                      # Select AP and PA encoded fieldmaps; merge niftis
                      (sf, create_merge, [('fmapap', 'ap'),
                                          ('fmappa', 'pa')]),
                      # Drop bottom slice of nifi (had odd # slices)
                      (create_merge, drop, [('merged_file', 'in_file')]),
                      # Run topop across merged niftis
                      (drop, topup, [('roi_file', 'in_file')]),
                      (sf, topup, [('aps', 'encoding_file')]),
                      (topup, datasink, [
                       ('out_corrected', '1_Check_Unwarped.@par')]),
                      # Extract b0 image from nifti with topup applied
                      (topup, fslroi, [('out_corrected', 'in_file')]),
                      #Register T1 to b0 brain
                      (sf, register1, [('t1', 'in_file')]),
                      (fslroi, register1, [('roi_file', 'reference')]),
                      #skullstrip T1
                      (register1, stripT1, [('out_file', 'in_file')]),
                      # Save stripped anat and mask
                      (stripT1, datasink, [('mask_file', '1_Check_Unwarped.@par.@par.@par.@par.@par.@par'),
                                           ('mask_file', '2_Preprocessed')]),
                      (stripT1, datasink, [('out_file', '1_Check_Unwarped.@par.@par.@par.@par.@par.@par.@par'),
                                             ('out_file', '2_Preprocessed.@par')]),
                      # Drop bottom slice from DTI nifti
                      (sf, drop2, [('dti', 'in_file')]),
                      # Local PCA to denoise DTI data
                      (drop2, denoise, [('roi_file', 'in_file')]),
                      (denoise, datasink, [
                       ('out_file', '1_Check_Unwarped.@par.@par.@par')]),
                      # Gibbs ringing removal
                      (drop2, gibbs, [('roi_file', 'in_file')]),
                      # Perform DWI bias field correction
                      (gibbs, bias, [('out_file', 'in_file')]),
                      (sf, bias, [('bvec', 'in_bvec')]),
                      (sf, bias, [('bval', 'in_bval')]),
                      # Apply topup to bias corrected DTI data
                      (topup, apptop, [('out_fieldcoef', 'in_topup_fieldcoef'),
                                      ('out_movpar','in_topup_movpar')]),
                      (bias, apptop, [('out_file', 'in_files')]),
                      (sf, apptop, [('aps', 'encoding_file')]),
                      (apptop, datasink, [
                          ('out_corrected', '1_Check_Unwarped.@par.@par.@par.@par.@par'),
                          ('out_corrected', '2_Preprocessed.@par.@par')])
                      ])
preproc_flow.base_dir = workflow_dir
preproc_flow.write_graph(graph2use='flat')
preproc = preproc_flow.run('MultiProc', plugin_args={'n_procs': 4})

190923-20:58:08,633 nipype.workflow DEBUG:
	 (preproc_flow.infosource, preproc_flow.sf): No edge data
190923-20:58:08,634 nipype.workflow DEBUG:
	 (preproc_flow.infosource, preproc_flow.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
190923-20:58:08,635 nipype.workflow DEBUG:
	 (preproc_flow.sf, preproc_flow.create_merge): No edge data
190923-20:58:08,635 nipype.workflow DEBUG:
	 (preproc_flow.sf, preproc_flow.create_merge): new edge data: {'connect': [('fmapap', 'ap'), ('fmappa', 'pa')]}
190923-20:58:08,636 nipype.workflow DEBUG:
	 (preproc_flow.create_merge, preproc_flow.drop): No edge data
190923-20:58:08,637 nipype.workflow DEBUG:
	 (preproc_flow.create_merge, preproc_flow.drop): new edge data: {'connect': [('merged_file', 'in_file')]}
190923-20:58:08,637 nipype.workflow DEBUG:
	 (preproc_flow.drop, preproc_flow.topup): No edge data
190923-20:58:08,638 nipype.workflow DEBUG:
	 (preproc_flow.drop, preproc_flow.topup): new edge data: {'connect': [('roi_file', 'in_file

190923-20:58:11,280 nipype.workflow DEBUG:
	 [MultiProc] Starting (n_procs=4, mem_gb=28.80, cwd=/Users/lucindasisk/Dropbox/Github/Diffusion)
190923-20:58:11,282 nipype.workflow DEBUG:
	 Creating flat graph for workflow: preproc_flow
190923-20:58:11,288 nipype.workflow DEBUG:
	 expanding workflow: preproc_flow
190923-20:58:11,290 nipype.workflow DEBUG:
	 processing node: preproc_flow.infosource
190923-20:58:11,290 nipype.workflow DEBUG:
	 processing node: preproc_flow.sf
190923-20:58:11,291 nipype.workflow DEBUG:
	 processing node: preproc_flow.drop2
190923-20:58:11,292 nipype.workflow DEBUG:
	 processing node: preproc_flow.gibbs
190923-20:58:11,292 nipype.workflow DEBUG:
	 processing node: preproc_flow.bias
190923-20:58:11,293 nipype.workflow DEBUG:
	 processing node: preproc_flow.denoise
190923-20:58:11,294 nipype.workflow DEBUG:
	 processing node: preproc_flow.resampT1
190923-20:58:11,294 nipype.workflow DEBUG:
	 processing node: preproc_flow.create_merge
190923-20:58:11,295 nipype.w

	 [Node] Up-to-date cache found for "preproc_flow.sf".
190923-20:58:14,282 nipype.workflow DEBUG:
	 [Node] Rerunning cached, up-to-date node "preproc_flow.sf"
190923-20:58:14,356 nipype.utils DEBUG:
	 Removing contents of /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/sf
190923-20:58:15,336 nipype.utils DEBUG:
	 Creating directory /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A201/sf/_report
190923-20:58:15,509 nipype.workflow DEBUG:
	 [Node] Writing pre-exec report to "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A201/sf/_report/report.rst"
190923-20:58:15,676 nipype.utils DEBUG:
	 Creating directory /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/sf/_report
190923-20:58:15,772 nipype.workflow DEBUG:
	 [Node] Writing pre-exec rep

190923-20:58:18,108 nipype.workflow DEBUG:
	 [Node] Up-to-date cache found for "preproc_flow.drop2".
190923-20:58:18,109 nipype.workflow DEBUG:
	 Checking hash "preproc_flow.drop2" locally: cached=True, updated=True.
190923-20:58:18,110 nipype.workflow DEBUG:
	 Skipping cached node preproc_flow.drop2 with ID 1.
190923-20:58:18,111 nipype.workflow INFO:
	 [Job 1] Cached (preproc_flow.drop2).
190923-20:58:18,113 nipype.workflow DEBUG:
	 Allocating preproc_flow.resampT1 ID=5 (0.20GB, 1 threads). Free: 28.40GB, 2 threads.
190923-20:58:18,113 nipype.workflow DEBUG:
	 Setting node inputs
190923-20:58:18,114 nipype.workflow DEBUG:
	 input: in_file
190923-20:58:18,115 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A201/sf/result_sf.pklz
190923-20:58:18,115 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_

190923-20:58:18,448 nipype.workflow DEBUG:
	 Allocating preproc_flow.drop2 ID=15 (0.20GB, 1 threads). Free: 28.00GB, 0 threads.
190923-20:58:18,449 nipype.workflow DEBUG:
	 Setting node inputs
190923-20:58:18,449 nipype.workflow DEBUG:
	 input: in_file
190923-20:58:18,450 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/sf/result_sf.pklz
190923-20:58:18,451 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/sf/result_sf.pklz
190923-20:58:18,474 nipype.workflow DEBUG:
	 output: dti
190923-20:58:18,475 nipype.workflow DEBUG:
	 [Node] drop2 - setting input in_file = /Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/dwi/sub-A200_ses-shapesV1_dwi.nii.gz
190923-20:58:18,474 nipype.utils DEBUG:
	 Removing contents of /Users/lucindasisk/Desktop/Milgram/c

190923-20:58:20,117 nipype.workflow DEBUG:
	 [MultiProc] Submitted task preproc_flow.denoise (taskid=5).
190923-20:58:20,118 nipype.workflow DEBUG:
	 Allocating preproc_flow.drop ID=7 (0.20GB, 1 threads). Free: 28.00GB, 0 threads.
190923-20:58:20,119 nipype.workflow DEBUG:
	 Setting node inputs
190923-20:58:20,119 nipype.workflow DEBUG:
	 input: in_file
190923-20:58:20,120 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A201/create_merge/result_create_merge.pklz
190923-20:58:20,121 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A201/create_merge/result_create_merge.pklz
190923-20:58:20,118 nipype.workflow INFO:
	 [Node] Setting-up "preproc_flow.denoise" in "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A201/denoise".190923-20:58:2

RuntimeError: Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/site-packages/nipype/pipeline/plugins/multiproc.py", line 69, in run_node
    result['result'] = node.run(updatehash=updatehash)
  File "/usr/local/anaconda3/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 410, in run
    cached, updated = self.is_cached()
  File "/usr/local/anaconda3/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 296, in is_cached
    hashed_inputs, hashvalue = self._get_hashval()
  File "/usr/local/anaconda3/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 493, in _get_hashval
    self._get_inputs()
  File "/usr/local/anaconda3/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 520, in _get_inputs
    outputs = loadpkl(results_file).outputs
  File "/usr/local/anaconda3/lib/python3.6/site-packages/nipype/utils/filemanip.py", line 668, in loadpkl
    raise e
  File "/usr/local/anaconda3/lib/python3.6/site-p

190923-20:59:26,368 nipype.workflow DEBUG:
	 Needed files: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A201/resampT1/sub-A201_ses-shapesV1_T1w_resample.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A201/ses-shapesV1/anat/sub-A201_ses-shapesV1_T1w.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A201/resampT1/_0x28b12dac9f88d2cca6a1b57de502fd80_unfinished.json;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A201/resampT1/command.txt;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A201/resampT1/_inputs.pklz;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A201/resampT1/_node.pklz
190923-20:59:26,373 nipype.workflow DEBUG:
	 Needed dirs: /Users/l